In [4]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()
model = ChatOpenAI()

In [5]:
class blogState(TypedDict):
  title: str
  outline: str
  content: str
  rating: str

In [6]:
def create_outline(state: blogState) -> blogState:
  title = state['title']

  prompt = f'Create a outline about the topic {title}, because later on i want to write a blog based on the outline.'

  outline = model.invoke(prompt).content
  state['outline'] = outline

  return state

def create_blog(state: blogState) -> blogState:
  title = state['title']
  outline = state['outline']

  prompt = f'Write a blog about the topic {title}, here is the outline about the topic -  \n {outline}'

  state['content'] = model.invoke(prompt).content

  return state

def evaluate_blog(state: blogState) -> blogState:
  title = state['title']
  content = state['content']

  prompt = f'Rate the blog between 1 to 10, please return only the rating number. topic of the blog {title}. Here is the blog - {content}'

  state['rating'] = model.invoke(prompt).content

  return state

In [8]:
graph = StateGraph(blogState)

graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)
graph.add_node('evaluate_blog', evaluate_blog)

graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', 'evaluate_blog')
graph.add_edge('evaluate_blog', END)

workflow = graph.compile()

In [10]:
initial_state = {'title' : 'Rising of AI in the world.'}
final_state = workflow.invoke(initial_state)

print(final_state['content'])

With the rapid advancements in technology, Artificial Intelligence (AI) has emerged as a major player in shaping the future of our world. From healthcare to finance, AI is revolutionizing various industries and transforming the way we work and live. In this blog, we will delve into the rising importance of AI in the world and explore its history, applications, benefits, challenges, and future prospects.

I. Introduction

A. Definition of AI: AI refers to the development of computer systems that can perform tasks that typically require human intelligence, such as visual perception, speech recognition, decision-making, and language translation.

B. Importance of AI in modern society: AI is increasingly being integrated into various aspects of daily life, from virtual assistants like Siri and Alexa to self-driving cars and personalized recommendations on streaming platforms. Its ability to analyze massive amounts of data and perform complex tasks has made it a valuable tool for businesses